In [31]:
import time
import random
# import pyepo
import torch
from torch import nn
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset

import sys
root_dir = "../../"
sys.path.append(root_dir)
from src.torch_Dijkstra import Dijkstra
from src.dys_opt_net import DYS_opt_net
from src.utils import node_to_edge

In [32]:
warcraft_data_folder = root_dir + './src/warcraft/warcraft_data/'

tmaps_train = np.load(warcraft_data_folder + "12x12/train_maps.npy")
tmaps_val = np.load(warcraft_data_folder + "12x12/val_maps.npy")
tmaps_test = np.load(warcraft_data_folder + "12x12/test_maps.npy")

# convert to Float Tensor and permute so that the channels are first
tmaps_train = torch.FloatTensor(tmaps_train).permute(0, 3, 1, 2)
tmaps_val = torch.FloatTensor(tmaps_val).permute(0, 3, 1, 2)
tmaps_test = torch.FloatTensor(tmaps_test).permute(0, 3, 1, 2)

true_cost_train = np.load(warcraft_data_folder + "12x12/train_vertex_weights.npy")
true_cost_val = np.load(warcraft_data_folder + "12x12/val_vertex_weights.npy")
true_cost_test = np.load(warcraft_data_folder + "12x12/test_vertex_weights.npy")

# convert to Float Tensor 
true_cost_train = torch.FloatTensor(true_cost_train)
true_cost_val = torch.FloatTensor(true_cost_val)
true_cost_test = torch.FloatTensor(true_cost_test)

# true paths
true_path_train = np.load(warcraft_data_folder + "12x12/train_shortest_paths.npy")
true_path_val = np.load(warcraft_data_folder + "12x12/val_shortest_paths.npy")
true_path_test = np.load(warcraft_data_folder + "12x12/test_shortest_paths.npy")

# convert to Float Tensor 
true_path_train_vertex = torch.FloatTensor(true_path_train)
true_path_val_vertex = torch.FloatTensor(true_path_val)
true_path_test_vertex = torch.FloatTensor(true_path_test)

In [33]:
 # Construct vertices. Note each vertex is at the middle of the square.
m = 12 # grid size
vertices = []
for i in range(m):
    for j in range(m):
        vertices.append((i+0.5, j+0.5))

num_vertices = len(vertices)

# Construct edges.
edge_list = []
for i, v1 in enumerate(vertices):
    for j, v2 in enumerate(vertices):
        norm_squared = (v1[0] - v2[0])**2 + (v1[1] - v2[1])**2
        if 0 < norm_squared < 2.01 and i < j:
            edge_list.append((v1, v2))
            edge_list.append((v2, v1)) # have to double edges to allow for travel both ways

num_edges = len(edge_list)
print('number of edges = ', num_edges)

# ## Small utility for rounding coordinates of points
# def round_coordinates(vertex_name):
#   vertex_coord = [int(vertex_name[0]), int(vertex_name[1])]
#   return vertex_coord

## vertex-edge incidence matrix
A = torch.zeros((len(vertices), len(edge_list)))
for j, e in enumerate(edge_list):
    ind0 = vertices.index(e[0])
    ind1 = vertices.index(e[1])
    A[ind0,j] = -1.
    A[ind1,j] = +1.

## Create b vector necessary for LP approach to shortest path
b = torch.zeros(m**2)
b[0] = -1.0
b[-1] = 1.0

number of edges =  1012


### Get paths in edge format as well

In [34]:
true_path_train_edge = node_to_edge(true_path_train_vertex, edge_list, four_neighbors=False)
true_path_val_edge = node_to_edge(true_path_val_vertex, edge_list, four_neighbors=False)
true_path_test_edge = node_to_edge(true_path_test_vertex, edge_list, four_neighbors=False)

### Create Tensor Datasets

In [35]:
train_dataset = TensorDataset(tmaps_train, true_path_train_edge, true_path_train_vertex, true_cost_train)
val_dataset = TensorDataset(tmaps_val, true_path_val_edge, true_path_val_vertex, true_cost_val)
test_dataset = TensorDataset(tmaps_test, true_path_test_edge, true_path_test_vertex, true_cost_test)

In [36]:
# Create a dictionary of the PyTorch variables
state_dict = {
'train_dataset': train_dataset,
'val_dataset': val_dataset,
'test_dataset': test_dataset,
'm': 12, # vertex grid size
'A': A,
'b': b,
'num_edges': num_edges,
'edge_list': edge_list
}

save_path = warcraft_data_folder + "Warcraft_training_data12.pth"

# Save the dictionary to a file
torch.save(state_dict, save_path)

In [27]:
# train_dataset_v = TensorDataset(tmaps_train, true_path_train, true_cost_train)
# true_path_train_edge = node_to_edge(true_path_train, edges, four_neighbors=False)
# train_dataset_e = TensorDataset(tmaps_train, true_path_train_edge, true_path_train_vertex, true_cost_train)

In [28]:
# val_dataset_v = TensorDataset(tmaps_val, true_path_val, true_cost_val)
# true_path_val_e = node_to_edge(true_path_val, edges, four_neighbors=False)
# val_dataset_e = TensorDataset(tmaps_val, true_path_val_e, true_cost_val)

In [29]:
# test_dataset_v = TensorDataset(tmaps_test, true_path_test, true_cost_test)
# true_path_test_e = node_to_edge(true_path_test, edges, four_neighbors=False)
# test_dataset_e = TensorDataset(tmaps_test, true_path_test_e, true_cost_test)

In [30]:
# # Create a dictionary of the PyTorch variables
# state_dict = {
# 'train_dataset_v': train_dataset_v,
# 'train_dataset_e': train_dataset_e,
# 'val_dataset_v': val_dataset_v,
# 'val_dataset_e': val_dataset_e,
# 'test_dataset_v': test_dataset_v,
# 'test_dataset_e': test_dataset_e,
# 'm': 12, # vertex grid size
# 'A': A,
# 'b': b,
# 'num_edges': num_edges,
# 'edge_list': edges
# }

# save_path = warcraft_data_folder + "Warcraft_training_data12.pth"

# # Save the dictionary to a file
# torch.save(state_dict, save_path)